# Modules

In [ ]:
import sys; sys.path.append("../modules")
from nanoword import *

import pprint

# writhe polynomial vs ab polynomials

### nanoword の生成
7交点(classical)の nanoword をランダムに作成。

In [ ]:
mnw = Nanoword.random_generator(7)
display(mnw.word, mnw.alphabet)

### Ind vs self-linking
各交点の ind と self-linking を計算し比較。

In [ ]:
for ltr in mnw.alphabet:
    print(f"{ltr}  -->  ind: {mnw.ind(ltr):>2}  /  sl: {mnw.self_linking(ltr)}")

print("\n-----\n")
print(f"writhe poly: {mnw.writhe_polynomial()}")
print(f"ab polys:    {mnw.ab_polynomials()}")

writhe poly が自明でも、ab polys が自明でないことがある。

In [ ]:
Count, data = 0, []
while len(data) < 1 and Count < 1000:
    mnw = Nanoword.random_generator(7)
    wp, abps = mnw.writhe_polynomial(), mnw.ab_polynomials()
    if wp == '(0)' and abps != ['',''] and len(abps[0]) > 10:
        data.append(mnw)
    Count += 1
for nw in data: print(f"{nw.word} --> ab polys: {nw.ab_polynomials()}")

### ランダムに計算

In [ ]:
Count = 0
data = []
while Count < 10:
    nw = Nanoword.random_generator(9)
    wp, abps = nw.writhe_polynomial(), nw.ab_polynomials()
    print(f"{nw.word}\n -->   wp: {wp}\n --> abps: {abps}")
    Count += 1

# Scratch

In [ ]:
mnw = Nanoword.random_generator(5)
mnw_rmi = R.I(mnw, reverse=True)
mnw_rmii = R.II(mnw, reverse=True)
mnw_rmiii = R.III(mnw)
print(mnw, [f"{l}" for l in mnw.alphabet])
#print(mnw, mnw_rmi_inv, mnw_rmii_inv)
#
for ltr in mnw_rmii.alphabet:
    for nw in [mnw, mnw_rmi, mnw_rmii, mnw_rmiii]:
        if nw is not None:
            print(f"{ltr} --self_linking--> {nw.self_linking(ltr)}")
    print("-----")

## n-writhe

In [ ]:
mnw = Nanoword("ABCDACEEBD", [Letter('A','a+'),Letter('B','b-'),Letter('C','b-'),Letter('D','b-'),Letter('E','b+')]) #Nanoword.random_generator(4)
print(mnw)

print([f"{l}, {s}" for l,s in mnw.signs_on_word()])

alph_w_ind=[{'letter': ltr, 'ind': mnw.ind(ltr)} for ltr in mnw.alphabet]; print(alph_w_ind)

J = [{'n': ind, 'c': 0} for ind in set([x['ind'] for x in alph_w_ind])]
for elm in J:
    for d in alph_w_ind:
        if d['ind'] == elm['n']:
            elm['c'] += d['letter'].sign.pm
if len([d for d in J if d['n'] == 0]) == 0:
    J.append({'n': 0, 'c': 0})
print(J)

wp = ""
for d in sorted(J, key=lambda x: x['n'], reverse=True):
    if not d['n'] == 0:
        wp += f"+({d['c']})t^{d['n']}"
    else:
        s = sum([d['c'] for d in J if not d['n'] == 0])
        wp += f"+({-s})"
        
print(wp[1:])
print()

In [ ]:
mnw.ab_polynomials()

## Drawing Gauss diagrams

In [ ]:
import math
import plotly.graph_objects as go
fig = go.Figure()

nw = Nanoword.random_generator(7) #('ABCDBACD', [Letter('A','a+'), Letter('B','b+'), Letter('C','a-'), Letter('D','a-')])  #
s = int(nw.size/2)
#------------b

# Set axes properties
fig.update_xaxes(range=[-0.2, 2.6], zeroline=False)
fig.update_yaxes(range=[-0.2, 2.6])

points_data = []
for i, c in enumerate(nw.word):
    points_data.append({'letter': [l for l in nw.alphabet if l.char == c].pop(), 
                   'coord': (math.cos(i*math.pi/s)+1.2, math.sin(i*math.pi/s)+1.2)})
fig.add_trace(
    go.Scatter(
        x = [p['coord'][0] for p in points_data],
        y = [p['coord'][1] for p in points_data],
        mode="markers", marker=dict(size=8.0, color="RoyalBlue"),
        showlegend=False,
        # text=[p['letter'].char for p in points_data],
    ))
def margin(tup):
    x, y = tup
    return (0.15*(x-1.2), 0.15*(y-1.2))    
fig.add_trace(
    go.Scatter(
        x = [p['coord'][0]+margin(p['coord'])[0] for p in points_data],
        y = [p['coord'][1]+margin(p['coord'])[1] for p in points_data],
        text=[p['letter'] for p in points_data],
        mode="text", showlegend=False,
    ))

paths = []
for ltr in nw.alphabet:
    coords = [p['coord'] for p in points_data if p['letter'] == ltr]
    paths.append(dict(type="path",
                     path="M {},{} Q 1.2,1.2 {},{}".format(
                         coords[0][0],coords[0][1],coords[1][0], coords[1][1]),
                     line_color="orange",))
else: fig.update_layout(shapes=paths)
        
fig.add_shape(type="circle",
    xref="x", yref="y",
    x0=0.2, y0=0.2, x1=2.2, y1=2.2,
    line_color="LightSeaGreen",
)

# Set figure size
fig.update_layout(width=600, height=600)

print(nw)
fig.update_layout(plot_bgcolor="white")
fig.update_xaxes(showgrid=False,showticklabels=False)
fig.update_yaxes(showgrid=False,showticklabels=False)
fig.update_layout(margin=dict(t=0, b=0, l=0, r=0))
fig.show()

In [ ]:
type(fig)

In [ ]:
nw = Nanoword('ABCBAC', [Letter('A','a+'), Letter('B','b+'), Letter('C','a-')])  #.random_generator(3)
alph = nw.alphabet

alph_posi = [{'ltr': ltr, 'positions': tuple([i for i, x in enumerate(nw.word) if x == ltr.char])}  for ltr in alph] 

s = int(nw.size/2)
points = []
for i, c in enumerate(nw.word):
    points.append({'letter': [l for l in nw.alphabet if l.char == c].pop(), 
                   'coordinate': (math.cos(i*math.pi/s)+1.2, math.sin(i*math.pi/s)+1.2)})
pprint.pprint(points)

## With Sympy

In [ ]:
from sympy import *
init_printing()

t = symbols('t')

In [ ]:
wp = sum([d['c']*(t**d['n']-1) for d in J])

display(wp)

In [ ]:
random.randint(0,1)

# Drawing virtual knot diagrams

In [ ]:
nw = Nanoword.random_generator(3) #('ABCBCA', [Letter('A','a+'), Letter('B','b+'), Letter('C','a-')]) #
print(nw, nw.alphabet); diag = Diagram(nw)

marr = diag.matrix
alph = nw.alphabet
ltrs = nw.letters_on_word
s = nw.size // 2
rng = list(range(s))

count = {ltr.char: 0 for ltr in ltrs}
for j, ltr in enumerate(ltrs):
    ltr_next = ltrs[(j+1)%nw.size]
    ind, ind_next = alph.index(ltr), alph.index(ltr_next)
    dff = ind_next-ind
    is_a, is_next_a = (ltr.sign.ab == 'a'), (ltr_next.sign.ab)
    #---
    if count[ltr.char] == 0:
        count[ltr.char] += 1
        if dff == 0:
            diag.create_route(start = (ind, 'r'), goal = (ind, 'b' if is_a else 't'))
        else:
            if count[ltr_next.char] == 0:
                diag.create_route(start = (ind, 'r'), goal = (ind_next, 'l'))
            # else:
            #     diag.create_route(start = (ind, 'r'), goal = (ind_next, 'b' if is_next_a else 't'))
    else:
        if dff == 0:
            diag.create_route(start = (ind, 't' if is_a else 'b'), goal = (ind, 'l'))
        else: pass
            
print(marr); diag.show()

In [ ]:
cc = Cell(4,5)
marr[cc.move('right')] = 10
print(marr)

In [ ]:
ab = 'a'
bin(0b111 ^ (ab == 'b'))

In [ ]:
bin(0b1110 | 0b1101)

In [ ]:
ab = 'b'
bin((0b0 if ab == 'a' else 0b10) | True)

In [ ]:
print(marr)
marr[(2,2)]